In [3]:
import torch
import csv
import os
import random
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, random_split
import pandas as pd
from dataset_reader import Traces_Dataset
from mlp_model import MLP
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import optuna
from optuna.trial import TrialState

In [53]:
DEVICE = torch.device("cpu")
target_features = 7
DIR = os.getcwd()
EPOCHS = 50

N_TRAIN_EXAMPLES = 10 * 300
N_VALID_EXAMPLES = 10 * 100

In [52]:
def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 5)
    layers = []

    in_features = 321
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 16, 321)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))
        in_features = out_features

    layers.append(nn.Linear(in_features, target_features))
    # layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)

In [61]:
def get_dataset(trial):
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128, 256, 512, 1024])
    
    dataset = Traces_Dataset('dataset_test.csv')
    dataset.split_dataset(0.95, 0.05, 0)
    dataset.clean_features()
    dataset.find_mean_std()
    dataset.normalize()
    print(dataset.inputs.shape)

    # initialize train, val, test set
    X_train = dataset[dataset.train_set.indices][0]
    Y_train = dataset[dataset.train_set.indices][1]

    X_val = dataset[dataset.val_set.indices][0]
    Y_val = dataset[dataset.val_set.indices][1]

    X_test = dataset[dataset.test_set.indices][0]
    Y_test = dataset[dataset.test_set.indices][1]

    # initialize dataloader 
    train_dataset = TensorDataset(X_train, Y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    val_dataset = TensorDataset(X_val, Y_val)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

    return train_loader, val_loader

In [64]:
def objective(trial):
    # Generate the model.
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    
    

    # Get the FashionMNIST dataset.
    train_loader, valid_loader = get_dataset(trial)

    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (train_inputs, train_targets) in enumerate(train_loader):
            # Limiting training data for faster epochs.
            if batch_idx * 10 >= N_TRAIN_EXAMPLES:
                break

            # data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
            train_inputs, train_targets = train_inputs.to(DEVICE), train_targets.to(DEVICE)

            optimizer.zero_grad()
            train_outputs = model(train_inputs)
            loss = nn.MSELoss()(train_outputs, train_targets)
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        total_val_loss = 0.0
        with torch.no_grad():
            for batch_idx, (val_inputs, val_targets) in enumerate(valid_loader):
                # Limiting validation data.
                if batch_idx * 10 >= N_VALID_EXAMPLES:
                    break
                # data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                val_inputs, val_targets = val_inputs.to(DEVICE), val_targets.to(DEVICE)

                val_outputs = model(val_inputs)
                # Get the index of the max log-probability.
                val_loss = nn.MSELoss()(val_outputs, val_targets)
                total_val_loss += val_loss.item()

            # Average validation loss for the epoch
        avg_val_loss = total_val_loss / len(valid_loader)

        trial.report(avg_val_loss, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return avg_val_loss

In [65]:
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=20, timeout=600)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2024-03-27 01:37:49,481] A new study created in memory with name: no-name-527fdee5-c8a9-490d-97aa-79f1fcf1484b


torch.Size([1000, 321])


[I 2024-03-27 01:37:49,780] Trial 0 finished with value: 2855.56884765625 and parameters: {'n_layers': 1, 'n_units_l0': 106, 'dropout_l0': 0.22733226711445575, 'optimizer': 'RMSprop', 'lr': 0.00010700405515314282, 'batch_size': 512}. Best is trial 0 with value: 2855.56884765625.


torch.Size([1000, 321])


[I 2024-03-27 01:37:50,045] Trial 1 finished with value: 148.9717559814453 and parameters: {'n_layers': 2, 'n_units_l0': 43, 'dropout_l0': 0.4990525363955867, 'n_units_l1': 22, 'dropout_l1': 0.2606691756103567, 'optimizer': 'Adam', 'lr': 0.04561459107659196, 'batch_size': 256}. Best is trial 1 with value: 148.9717559814453.


...

torch.Size([1000, 321])
torch.Size([1000, 321])


[I 2024-03-27 01:38:03,378] Trial 18 finished with value: 49.37183094024658 and parameters: {'n_layers': 4, 'n_units_l0': 161, 'dropout_l0': 0.20600770445362854, 'n_units_l1': 150, 'dropout_l1': 0.30167377439845716, 'n_units_l2': 267, 'dropout_l2': 0.49784224537991045, 'n_units_l3': 110, 'dropout_l3': 0.2690332022158084, 'optimizer': 'Adam', 'lr': 0.0006726580394291194, 'batch_size': 16}. Best is trial 18 with value: 49.37183094024658.


torch.Size([1000, 321])


[I 2024-03-27 01:38:07,775] Trial 19 finished with value: 45.33682465553284 and parameters: {'n_layers': 4, 'n_units_l0': 174, 'dropout_l0': 0.20698929462045085, 'n_units_l1': 225, 'dropout_l1': 0.30723814508389546, 'n_units_l2': 321, 'dropout_l2': 0.4999956412579912, 'n_units_l3': 107, 'dropout_l3': 0.2550368762103258, 'optimizer': 'Adam', 'lr': 0.0003417690685853737, 'batch_size': 16}. Best is trial 19 with value: 45.33682465553284.


Study statistics: 
  Number of finished trials:  20
  Number of pruned trials:  9
  Number of complete trials:  11
Best trial:
  Value:  45.33682465553284
  Params: 
    n_layers: 4
    n_units_l0: 174
    dropout_l0: 0.20698929462045085
    n_units_l1: 225
    dropout_l1: 0.30723814508389546
    n_units_l2: 321
    dropout_l2: 0.4999956412579912
    n_units_l3: 107
    dropout_l3: 0.2550368762103258
    optimizer: Adam
    lr: 0.0003417690685853737
    batch_size: 16


In [66]:
optuna.importance.get_param_importances(study)

{'batch_size': 0.5167787332586548,
 'n_layers': 0.16080507562249335,
 'optimizer': 0.15665348579823324,
 'dropout_l0': 0.06870345193655863,
 'lr': 0.06637936426822885,
 'n_units_l0': 0.0306798891158311}